<h1>Alexnet Majority Vote</h1>
We start by recreating the model used in the orgininal MRNet paper to get a baseline for performance
This model uses three Alexnet backbones with a dense classification layer trained on axial, coronal and sagittal MRIs respectively and then uses a majority vote system to determine the final output.

We found the best parameters to be (parameters). As in the original paper, each sample was randomly rotated by an angle between -25 adn 25 degrees, randomly translated by up to 25 pixels and flipped horizontaly with probability 50%

Note for write up: didn't go in depth here since I didn't do it, please add some more details

<h3>Model class</h3>

In [18]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision.models import AlexNet_Weights

class MRNet3(nn.Module):
    
    def __init__(self,use_batchnorm=False):
        super().__init__()
        self.model1 = models.alexnet(weights=AlexNet_Weights.DEFAULT)
        self.model2 = models.alexnet(weights=AlexNet_Weights.DEFAULT)
        self.model3 = models.alexnet(weights=AlexNet_Weights.DEFAULT)
        self.gap = nn.AdaptiveMaxPool2d(1)
        # self.gap = nn.AdaptiveAvgPool2d(1)
        self.use_batchnorm = use_batchnorm
        n = 0.15
        # Dropout for each view's features
        self.dropout_view1 = nn.Dropout(p=n)
        self.dropout_view2 = nn.Dropout(p=n)
        self.dropout_view3 = nn.Dropout(p=n)
        
        print(f"Dropout of {n}")


        classifier_layers_axial = [nn.Linear(256, 256)]
        if self.use_batchnorm:
            classifier_layers_axial.append(nn.BatchNorm1d(256))
        self.classifier1_axial = nn.Sequential(*classifier_layers_axial)

        classifier_layers_coronal = [nn.Linear(256, 256)]
        if self.use_batchnorm:
            classifier_layers_coronal.append(nn.BatchNorm1d(256))
        self.classifier1_coronal = nn.Sequential(*classifier_layers_coronal)

        classifier_layers_sagittal = [nn.Linear(256, 256)]
        if self.use_batchnorm:
            classifier_layers_sagittal.append(nn.BatchNorm1d(256))
        self.classifier1_sagittal = nn.Sequential(*classifier_layers_sagittal)


        # Separate classifier2 for each view
        self.classifier2_axial = nn.Linear(256, 1)
        self.classifier2_coronal = nn.Linear(256, 1)
        self.classifier2_sagittal = nn.Linear(256, 1)


    #New forward pass to deal with batch normalisation

    def forward(self, x): 

        # Separate by view
        axial_views    = [sample[0] for sample in x]
        coronal_views  = [sample[1] for sample in x]
        sagittal_views = [sample[2] for sample in x]

        def process_view(view_list, model, dropout, classifier1, classifier2):
            features = []
            for view in view_list:
                slices, c, h, w = view.size()  # [num_slices, 3, 224, 224]
                view = view.view(slices, c, h, w).to(next(model.parameters()).device)
                feat = model.features(view)                     # [slices, 256, 6, 6]
                feat = self.gap(feat).view(slices, 256)         # [slices, 256]
                feat = torch.max(feat, dim=0)[0]                # [256]
                feat = dropout(feat)
                features.append(feat)
            features = torch.stack(features)                    # [batch_size, 256]
            features = classifier1(features)                    # [batch_size, 256]
            logits = classifier2(features)                      # [batch_size, 1]
            return logits

        logit_axial    = process_view(axial_views,    self.model1, self.dropout_view1, self.classifier1_axial, self.classifier2_axial)
        logit_coronal  = process_view(coronal_views,  self.model2, self.dropout_view2, self.classifier1_coronal, self.classifier2_coronal)
        logit_sagittal = process_view(sagittal_views, self.model3, self.dropout_view3, self.classifier1_sagittal, self.classifier2_sagittal)

        logits = torch.stack([logit_axial, logit_coronal, logit_sagittal], dim=0)  # [3, batch_size, 1]
        probs = torch.sigmoid(logits)                                              # [3, batch_size, 1]
        majority_prob = torch.mean(probs, dim=0)                                   # [batch_size, 1]
    
        return majority_prob

<h3>Loader</h3>

In [24]:
import numpy as np
import os
import torch
import torch.nn.functional as F
import torch.utils.data as data
import pandas as pd
from sklearn.model_selection import train_test_split
import kornia.augmentation as K
import random

INPUT_DIM = 224
MAX_PIXEL_VAL = 255
MEAN = 58.09
STDDEV = 49.73

class Dataset3(data.Dataset):
    def __init__(self, data_dir, file_list, labels_dict, device, train=False, augment=False):
        super().__init__()
        self.device = device
        self.data_dir_axial = f"{data_dir}/axial"
        self.data_dir_coronal = f"{data_dir}/coronal"
        self.data_dir_sagittal = f"{data_dir}/sagittal"

        self.paths_axial = [os.path.join(self.data_dir_axial, file) for file in file_list]
        self.paths_coronal = [os.path.join(self.data_dir_coronal, file) for file in file_list]
        self.paths_sagittal = [os.path.join(self.data_dir_sagittal, file) for file in file_list]
        
        self.paths = [self.paths_axial, self.paths_coronal, self.paths_sagittal]
        
        self.labels = [labels_dict[file] for file in file_list]

        neg_weight = np.mean(self.labels)
        self.weights = [neg_weight, 1 - neg_weight]

        self.train = train  #this ensures even when augment = True we never perform data augmentation on the validation/test set
        self.augment = augment              

    def weighted_loss(self, prediction, target, eps: float = 0.0):
        # Ensure target is [batch_size, 1]
        target = target.view(-1, 1)

        # Compute weights for each sample
        weights_npy = np.array([self.weights[int(t.item())] for t in target.flatten()])

        # Reshape weights to [batch_size, 1] to match prediction and target
        weights_tensor = torch.FloatTensor(weights_npy).view(-1, 1).to(target.device)

        smoothed = target * (1 - eps) + (1 - target) * eps # new

        # 3) compute BCE with logits against the *smoothed* targets
        loss = F.binary_cross_entropy_with_logits(prediction, smoothed, weight=weights_tensor) #new
        return loss
        # # Compute loss with weights reshaped to [batch_size, 1]
        # loss = F.binary_cross_entropy_with_logits(prediction, target, weight=weights_tensor)

        # return loss

    def __getitem__(self, index):
        vol_list = []
        for i in range(3):           
            path = self.paths[i][index]
            vol = np.load(path).astype(np.int32)
            pad = int((vol.shape[2] - INPUT_DIM) / 2)
            vol = vol[:, pad:-pad, pad:-pad]
            vol = (vol - np.min(vol)) / (np.max(vol) - np.min(vol)) * MAX_PIXEL_VAL
            vol = (vol - MEAN) / STDDEV
            vol = np.stack((vol,) * 3, axis=1)
            vol_tensor = torch.FloatTensor(vol)  # Keep on CPU
            vol_list.append(vol_tensor)

            # Apply augmentations if train and augment flags are True
            if self.train and self.augment:
                vol_tensor = self.apply_augmentations(vol_tensor)
        
            vol_list.append(vol_tensor)

        label_tensor = torch.FloatTensor([self.labels[index]])  # Shape: [1]
        return vol_list, label_tensor
    
    def apply_augmentations(self, vol_tensor):
        # Apply same augmentations slice-wise
        vol_tensor = K.RandomRotation(degrees=25)(vol_tensor)
        vol_tensor = K.RandomAffine(degrees=0, translate=(25/224, 25/224))(vol_tensor)
        if random.random() > 0.5:
            vol_tensor = K.RandomHorizontalFlip(p=1.0)(vol_tensor)
        return vol_tensor
    

    def __len__(self):
        return len(self.labels)

def custom_collate_fn(batch):
    """
    Custom collate function to handle variable slice counts.
    Returns a list of view tensors and a stacked label tensor.
    """
    vol_lists = [item[0] for item in batch]  # List of [axial, coronal, sagittal] for each sample
    labels = torch.stack([item[1] for item in batch], dim=0)  # Stack labels: [batch_size, 1]
    return vol_lists, labels

def load_data3(device, data_dir, labels_csv, batch_size=1, augment=False):
    labels_df = pd.read_csv(labels_csv, header=None, names=['filename', 'label'])
    labels_df['filename'] = labels_df['filename'].apply(lambda x: f"{int(x):04d}.npy")
    
    # Filter files that exist in all 3 views
    valid_files = []
    valid_labels = []
    for _, row in labels_df.iterrows():
        fname = row['filename']
        exists_all_views = all(os.path.exists(os.path.join(data_dir, view, fname)) for view in ['axial', 'coronal', 'sagittal'])
        if exists_all_views:
            valid_files.append(fname)
            valid_labels.append(row['label'])
    
    labels_dict = dict(zip(valid_files, valid_labels))

    # Stratify split
    train_files, valid_files = train_test_split(
        valid_files,
        test_size=0.2,
        random_state=42,
        stratify=valid_labels
    )

    train_dataset = Dataset3(data_dir, train_files, labels_dict, device, train=True, augment=augment)
    valid_dataset = Dataset3(data_dir, valid_files, labels_dict, device, train=False, augment=False)

    train_loader = data.DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        num_workers=0, 
        shuffle=True, 
        pin_memory=device.type == 'cuda',
        collate_fn=custom_collate_fn
    )

    valid_loader = data.DataLoader(
        valid_dataset, 
        batch_size=batch_size, 
        num_workers=0, 
        shuffle=False, 
        pin_memory=device.type == 'cuda',
        collate_fn=custom_collate_fn
    )

    print(f"Training samples: {len(train_dataset)}, Validation samples: {len(valid_dataset)}")
    return train_loader, valid_loader

def load_data_test(device, data_dir, labels_csv, batch_size=1):
    
    labels_df = pd.read_csv(labels_csv, header=None, names=['filename', 'label'])
    labels_df['filename'] = labels_df['filename'].apply(lambda x: f"{int(x):04d}.npy")
    
    # Filter files that exist in all 3 views
    test_files = []
    test_labels = []
    for _, row in labels_df.iterrows():
        fname = row['filename']
        exists_all_views = all(os.path.exists(os.path.join(data_dir, view, fname)) for view in ['axial', 'coronal', 'sagittal'])
        if exists_all_views:
            test_files.append(fname)
            test_labels.append(row['label'])
    
    labels_dict = dict(zip(test_files, test_labels))

    test_dataset = Dataset3(data_dir, test_files, labels_dict, device, train=False, augment=False)

    test_loader = data.DataLoader(
        test_dataset, 
        batch_size=batch_size, 
        num_workers=0, 
        shuffle=False, 
        pin_memory=device.type == 'cuda',
        collate_fn=custom_collate_fn
    )
    return test_loader


<h3>Training and evaluation functions</h3>

In [25]:
import argparse
import matplotlib.pyplot as plt
import os
import numpy as np
import torch
from sklearn import metrics
from torch.autograd import Variable
from tqdm import tqdm

def get_device(use_gpu, use_mps):
    
    if use_gpu and torch.cuda.is_available():
        return torch.device("cuda")
    
    elif use_mps and torch.backends.mps.is_available():
        return torch.device("mps")
    
    else:
        return torch.device("cpu")

def run_model(model, loader, train=False, optimizer=None, eps: float = 0.0):
    """
    model    : your MRNet3 instance
    loader   : DataLoader returning (vol_lists, label)
    train    : whether to do optimizer.step()
    optimizer: your Adam optimizer (only used if train=True)
    eps      : label-smoothing factor (0.0 = no smoothing)
    """
    preds = []
    labels = []
    total_loss = 0.0
    num_batches = 0

    if train:
        model.train()
    else:
        model.eval()

    device = loader.dataset.device

    for vol_lists, label in tqdm(loader, desc="Processing batches", total=len(loader)):
        # Move data to device
        label = label.to(device)                       # [batch_size,1]
        vol_lists = [[view.to(device) for view in views] for views in vol_lists]

        # Forward
        logits = model(vol_lists)                      # [batch_size,1]
        probs  = torch.sigmoid(logits)                 # [batch_size,1]

        # Loss
        if train:
            # use smoothing in training
            loss = loader.dataset.weighted_loss(logits, label, eps=eps)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        else:
            # no smoothing in val/test
            loss = loader.dataset.weighted_loss(logits, label, eps=0.0)

        # Accumulate
        total_loss += loss.item()
        preds.extend(probs.detach().cpu().view(-1).tolist())
        labels.extend(label.detach().cpu().view(-1).tolist())
        num_batches += 1

    avg_loss = total_loss / num_batches
    fpr, tpr, _ = metrics.roc_curve(labels, preds)
    auc = metrics.auc(fpr, tpr)

    return avg_loss, auc, preds, labels

def evaluate(split, model_path, use_gpu, mps, data_dir, labels_csv):
    device = get_device(use_gpu, mps)
    print(f"Using device: {device}")
    
    
    if split == 'train' or split == 'valid':
        train_loader, valid_loader = load_data3(device, data_dir, labels_csv)
    elif split == 'test':
        test_loader = load_data_test(device, data_dir, labels_csv)
    else:
        raise ValueError("split must be 'train', 'valid', or 'test'")
    
    model = MRNet3()
    
    state_dict = torch.load(model_path, map_location=device)
    model.load_state_dict(state_dict)
    model = model.to(device)
    
    if split == 'train':
        loader = train_loader
    elif split == 'valid':
        loader = valid_loader
    elif split == 'test':
        loader = test_loader

    loss, auc, preds, labels = run_model(model, loader, train=False)
    print(f'{split} loss: {loss:.4f}')
    print(f'{split} AUC: {auc:.4f}')
    return preds, labels


In [26]:
import argparse
import json
import numpy as np
import os
import torch
from datetime import datetime
from pathlib import Path
from sklearn import metrics

def get_device(use_gpu, use_mps):
    if use_gpu and torch.cuda.is_available():
        return torch.device("cuda")
    elif use_mps and torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")

def train3(rundir, epochs, learning_rate, gpu, mps, data_dir, labels_csv, weight_decay, max_patience, batch_size, augment, eps):
    device = get_device(gpu, mps)
    print(f"Using device: {device}")
    train_loader, valid_loader = load_data3(device, data_dir, labels_csv, batch_size=batch_size, augment=augment)
    
    #This now deals with the case that batch size is 1
    use_batchnorm = batch_size > 1
    model = MRNet3(use_batchnorm=use_batchnorm)
    model = model.to(device)

    print(f"Using BatchNorm: {use_batchnorm}")

    optimizer = torch.optim.Adam(model.parameters(), learning_rate, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=max_patience, factor=.3, threshold=1e-4)

    best_val_auc = float('-inf')

    start_time = datetime.now()

    epsilon = eps
    print(f"Value of eps:{epsilon}")
    for epoch in range(epochs):
        change = datetime.now() - start_time
        print('starting epoch {}. time passed: {}'.format(epoch+1, str(change)))
        
        train_loss, train_auc, _, _ = run_model(model, train_loader, train=True, optimizer=optimizer, eps=epsilon)
        print(f'train loss: {train_loss:0.4f}')
        print(f'train AUC: {train_auc:0.4f}')

        val_loss, val_auc, _, _ = run_model(model, valid_loader, eps=0.0)
        print(f'valid loss: {val_loss:0.4f}')
        print(f'valid AUC: {val_auc:0.4f}')

        scheduler.step(val_loss)

        if val_auc > best_val_auc:
            best_val_auc = val_auc
            file_name = f'val{val_auc:0.4f}_train{train_auc:0.4f}_epoch{epoch+1}'
            save_path = Path(rundir) / file_name 
            torch.save(model.state_dict(), save_path)

        # Log metrics to file
        with open(os.path.join(rundir, 'metrics.txt'), 'a') as f:
            f.write(f"Epoch {epoch+1}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, train_auc={train_auc:.4f}, val_auc={val_auc:.4f}\n")


<h3>Training</h3>

In [27]:
rundir =  "/Users/matteobruno/Desktop/runs"  #"directory/to/store/runs"
data_dir = "/Users/matteobruno/Desktop/models_and_data/MRNet-v1.0/train" #"Directory/containing/.npy_files'"
labels_csv =  "/Users/matteobruno/Desktop/models_and_data/MRNet-v1.0/train/train-acl.csv" #"Path/to/labels/CSV/file"
seed = 42
gpu = False #If true runs on Nvidia GPU
mps = True #If true runs on Apple MPS
learning_rate = 1e-05
weight_decay = 0.0025
epochs = 50
max_patience = 5
factor = 0.3 
batch_size = 1 
eps = 0.0 #Label smoothing factor (0.0 = no smoothing)'
augment = True  #Apply data augmentation during training


np.random.seed(seed)
torch.manual_seed(seed)
if gpu and torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
elif mps and torch.backends.mps.is_available():
    pass

os.makedirs(rundir, exist_ok=True)

# Save parameters to args.json
params = {
    "rundir": rundir,
    "data_dir": data_dir,
    "labels_csv": labels_csv,
    "seed": seed,
    "gpu": gpu,
    "mps": mps,
    "learning_rate": learning_rate,
    "weight_decay": weight_decay,
    "epochs": epochs,
    "max_patience": max_patience,
    "batch_size": batch_size,
    "label_smoothing": eps,
    "augment": augment
}
with open(Path(rundir) / 'args.json', 'w') as out:
    json.dump(params, out, indent=4)

    train3(rundir, epochs, learning_rate, 
        gpu, mps, data_dir, labels_csv, weight_decay, max_patience, batch_size, augment, eps)

Using device: mps
Training samples: 904, Validation samples: 226
Dropout of 0.15
Using BatchNorm: False
Value of eps:0.0
starting epoch 1. time passed: 0:00:00.000017


Processing batches:   2%|▏         | 17/904 [00:04<03:52,  3.82it/s]


KeyboardInterrupt: 

<h3>Testing</h3>

In [28]:
model_path = "/Users/matteobruno/Desktop/models_and_data/Best_alexnet_majority/best_model.pth"  # Path to the saved model
split = "test"  # or "train", "valid"
data_dir = "/Users/matteobruno/Desktop/models_and_data/MRNet-v1.0/test" #"Directory/containing/.npy_files'"
labels_csv =  "/Users/matteobruno/Desktop/models_and_data/MRNet-v1.0/test/valid-acl.csv" #"Path/to/labels/CSV/file"
gpu = False #If true runs on Nvidia GPU
mps = True #If true runs on Apple MPS


evaluate(split, model_path, gpu, mps, data_dir, labels_csv)

Using device: mps
Dropout of 0.15


Processing batches: 100%|██████████| 120/120 [00:18<00:00,  6.52it/s]

test loss: 0.3050
test AUC: 0.9495


([0.5079914331436157,
  0.501662015914917,
  0.534428596496582,
  0.5015807151794434,
  0.534299910068512,
  0.512273907661438,
  0.5107388496398926,
  0.6554552316665649,
  0.5007240176200867,
  0.5014246702194214,
  0.5126606822013855,
  0.5208551287651062,
  0.5003081560134888,
  0.5004255175590515,
  0.5149188041687012,
  0.5026782751083374,
  0.5003659725189209,
  0.5553420782089233,
  0.5008665323257446,
  0.5003727674484253,
  0.5336424112319946,
  0.5008522868156433,
  0.5007810592651367,
  0.5008218884468079,
  0.5069884657859802,
  0.5008457899093628,
  0.5152267217636108,
  0.5016493201255798,
  0.5043213963508606,
  0.50148606300354,
  0.5004546046257019,
  0.5328283309936523,
  0.5932879447937012,
  0.5320087671279907,
  0.5007878541946411,
  0.502911388874054,
  0.5011802315711975,
  0.5090853571891785,
  0.5007158517837524,
  0.5008416771888733,
  0.5638201236724854,
  0.5247063636779785,
  0.5924491286277771,
  0.5798209309577942,
  0.6506409049034119,
  0.5835934877395

<h1>Resnet</h1>
We then decided to try to use a more recent and advanced CNN as a backbone for our model. We opted for Resnet.
Other than switching from three Alexnet backbones to three Resnet backbones, we also made some other changes to improve AUC and training speed:

- The forward pass process all slices at once instead of using a for loop. Since views may have different number of slices, they are padded to the max number of slices in the sample and then a mask is used to consider only the origina slices. This allows for significantly faster trainign 

- Instead of using a majority vote sistem, the classification layer is a two layers deep MLP

The final version of the model has been trained with the following setup:

- No data augmentation, since it is computationally expensive and didn't seem to provide any significant benefit

- Strong dropout of 0.7 after feature extraction to prevent the significant overfetting that we were initially experiencing 

- Lable smoothing with a factor of 0.1 to further regularize the model 

- Batch size of 4 since it was the largest that could fit in memory

The other parameters can be seen in the training cell.

We obtained a validation AUC of 95.47% (train AUC of 99.74%) and a testing AUC of 97.5%, likely due to some luck and ot the small size of the sample test 

<h3>Model class</h3>

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision.models import ResNet18_Weights

class MRNet3(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Initialize ResNet18 backbones (already have BN internally)
        self.model1 = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        self.model2 = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        self.model3 = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        
        # Remove the original classification layer
        self.model1 = nn.Sequential(*list(self.model1.children())[:-1])
        self.model2 = nn.Sequential(*list(self.model2.children())[:-1])
        self.model3 = nn.Sequential(*list(self.model3.children())[:-1])

        self.gap = nn.AdaptiveAvgPool2d(1)  # Global Average Pooling
        
        # Dropout for each view's features
        self.dropout_view1 = nn.Dropout(p=0.7)
        self.dropout_view2 = nn.Dropout(p=0.7)
        self.dropout_view3 = nn.Dropout(p=0.7)
        
        # Fully connected layers with batch normalization
        self.classifier1 = nn.Linear(512 * 3, 256)  # Concatenated features from 3 views
        self.bn1 = nn.BatchNorm1d(256)  # BN after classifier1
        self.dropout = nn.Dropout(p=0.4)
        self.activation = nn.ReLU()
        self.classifier2 = nn.Linear(256, 1)

    def forward(self, x, original_slices):
        
        view_features = []
        
        for view in range(3):
            
            x_view = x[view]  # [B, S_max, 3, 224, 224]
            B, S_max, _, H, W = x_view.shape
            x_view = x_view.view(B * S_max, 3, H, W)
            
            if view == 0:
                features = self.model1(x_view)
            elif view == 1:
                features = self.model2(x_view)
            else:
                features = self.model3(x_view)
            
            features = self.gap(features).view(B, S_max, 512)  # [B, S_max, 512]
            s_indices = torch.arange(S_max, device=features.device).unsqueeze(0).expand(B, S_max)
            mask = s_indices < original_slices[view].unsqueeze(1)
            features = features.masked_fill(~mask.unsqueeze(2), -float('inf'))
            max_features = torch.max(features, dim=1)[0]  # [B, 512]
            
            if view == 0:
                max_features = self.dropout_view1(max_features)
            elif view == 1:
                max_features = self.dropout_view2(max_features)
            else:
                max_features = self.dropout_view3(max_features)
            
            view_features.append(max_features)
        
        # Concatenate features from all views
        x_stacked = torch.cat(view_features, dim=1)  # [B, 1536]
        
        # Fully connected layers with BN
        x_stacked = self.classifier1(x_stacked)  # [B, 256]
        x_stacked = self.bn1(x_stacked)  # Apply batch normalization
        x_stacked = self.dropout(x_stacked)
        x_stacked = self.activation(x_stacked)
        x_stacked = self.classifier2(x_stacked)  # [B, 1]
        
        return x_stacked

<h3>Loader</h3>

In [ ]:
import numpy as np
import os
import torch
import torch.nn.functional as F
import torch.utils.data as data
import pandas as pd
from sklearn.model_selection import train_test_split

INPUT_DIM = 224
MAX_PIXEL_VAL = 1.0  # ResNet expects [0, 1] before channel-wise normalization
MEAN = [0.485, 0.456, 0.406]  # ImageNet mean for ResNet (per channel)
STDDEV = [0.229, 0.224, 0.225]  # ImageNet std for ResNet (per channel)

class MRDataset(data.Dataset):
    def __init__(self, data_dir, file_list, labels_dict, device, label_smoothing=0.1):
        super().__init__()
        self.device = device
        self.data_dir_axial = f"{data_dir}/axial"
        self.data_dir_coronal = f"{data_dir}/coronal"
        self.data_dir_sagittal = f"{data_dir}/sagittal"

        self.paths_axial = [os.path.join(self.data_dir_axial, file) for file in file_list]
        self.paths_coronal = [os.path.join(self.data_dir_coronal, file) for file in file_list]
        self.paths_sagittal = [os.path.join(self.data_dir_sagittal, file) for file in file_list]
        
        self.paths = [self.paths_axial, self.paths_coronal, self.paths_sagittal]
        
        self.labels = [labels_dict[file] for file in file_list]
        self.label_smoothing = label_smoothing  # New parameter for label smoothing

        neg_weight = np.mean(self.labels)
        dtype = np.float32
        self.weights = [dtype(neg_weight), dtype(1 - neg_weight)]

    def weighted_loss(self, prediction, target, train):
        dtype = torch.float32
        indices = target.squeeze(1).long()  # Shape: [B]
        weights_tensor = torch.tensor(self.weights, device=self.device, dtype=dtype)[indices]  # Shape: [B]
        weights_tensor = weights_tensor.unsqueeze(1)  # Shape: [B, 1]

        # Apply label smoothing only during training if label_smoothing > 0
        if train and self.label_smoothing > 0:
            smoothed_target = target * (1 - self.label_smoothing) + (1 - target) * self.label_smoothing
        else:
            smoothed_target = target

        loss = F.binary_cross_entropy_with_logits(prediction, smoothed_target, weight=weights_tensor)
        return loss

    def __getitem__(self, index):
        vol_list = []

        for i in range(3):           
            path = self.paths[i][index]
            vol = np.load(path).astype(np.float32) 

            # Crop to INPUT_DIM x INPUT_DIM (224x224)
            pad = int((vol.shape[2] - INPUT_DIM) / 2)
            vol = vol[:, pad:-pad, pad:-pad]

            # Normalize to [0, 1]
            vol = (vol - np.min(vol)) / (np.max(vol) - np.min(vol) + 1e-6)  # [0, 1]

            # Stack to 3 channels
            vol = np.stack((vol,) * 3, axis=1)  # Shape: (slices, 3, 224, 224)

            # Apply ImageNet normalization per channel
            vol_tensor = torch.FloatTensor(vol).to(self.device)  # Shape: (slices, 3, 224, 224)
            for c in range(3):
                vol_tensor[:, c, :, :] = (vol_tensor[:, c, :, :] - MEAN[c]) / STDDEV[c]

            vol_list.append(vol_tensor)

        label_tensor = torch.FloatTensor([self.labels[index]]).to(self.device)

        return vol_list, label_tensor

    def __len__(self):
        return len(self.labels)

def collate_fn(batch):
    device = batch[0][0][0].device
    view0_list = [sample[0][0] for sample in batch]  # Axial
    view1_list = [sample[0][1] for sample in batch]  # Coronal
    view2_list = [sample[0][2] for sample in batch]  # Sagittal
    
    # Pad slices to the maximum in the batch for each view
    padded_view0 = torch.nn.utils.rnn.pad_sequence(view0_list, batch_first=True)
    padded_view1 = torch.nn.utils.rnn.pad_sequence(view1_list, batch_first=True)
    padded_view2 = torch.nn.utils.rnn.pad_sequence(view2_list, batch_first=True)
    
    # Store original slice counts for masking in the model
    original_slices0 = torch.tensor([v.shape[0] for v in view0_list], device=device)
    original_slices1 = torch.tensor([v.shape[0] for v in view1_list], device=device)
    original_slices2 = torch.tensor([v.shape[0] for v in view2_list], device=device)
    
    # Stack labels
    labels = torch.stack([sample[1] for sample in batch])
    
    return [padded_view0, padded_view1, padded_view2], labels, [original_slices0, original_slices1, original_slices2]

def load_data3(device, data_dir, labels_csv, batch_size=1, label_smoothing=0.1):
    labels_df = pd.read_csv(labels_csv, header=None, names=['filename', 'label'])
    labels_df['filename'] = labels_df['filename'].apply(lambda x: f"{int(x):04d}.npy")
    labels_dict = dict(zip(labels_df['filename'], labels_df['label']))

    all_files = [f for f in os.listdir(f"{data_dir}/axial") if f.endswith(".npy")]
    all_files = [f for f in all_files if f in labels_dict]
    all_files.sort()

    labels = [labels_dict[file] for file in all_files]

    train_files, valid_files = train_test_split(
        all_files, 
        test_size=0.2, 
        random_state=42, 
        stratify=labels
    )

    train_dataset = MRDataset(data_dir, train_files, labels_dict, device, label_smoothing=label_smoothing)
    valid_dataset = MRDataset(data_dir, valid_files, labels_dict, device, label_smoothing=label_smoothing)

    train_loader = data.DataLoader(train_dataset, batch_size=batch_size, num_workers=0, shuffle=True, collate_fn=collate_fn)
    valid_loader = data.DataLoader(valid_dataset, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=collate_fn)

    return train_loader, valid_loader

def load_data_test(device, data_dir, labels_csv, batch_size=1, label_smoothing=0):
    
    labels_df = pd.read_csv(labels_csv, header=None, names=['filename', 'label'])
    labels_df['filename'] = labels_df['filename'].apply(lambda x: f"{int(x):04d}.npy")
    labels_dict = dict(zip(labels_df['filename'], labels_df['label']))

    test_files = [f for f in os.listdir(f"{data_dir}/axial") if f.endswith(".npy")]
    test_files = [f for f in test_files if f in labels_dict]
    test_files.sort()

    test_dataset = MRDataset(data_dir, test_files, labels_dict, device, label_smoothing=label_smoothing)

    test_loader = data.DataLoader(test_dataset, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=collate_fn)

    return test_loader

<h3>Training and evaluation functions</h3>

In [ ]:
import argparse
import json
import numpy as np
import os
import torch

from datetime import datetime
from pathlib import Path
from sklearn import metrics

def get_device(use_gpu, use_mps):
    if use_gpu and torch.cuda.is_available():
        return torch.device("cuda")
    elif use_mps and torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")

def train3(rundir, epochs, learning_rate, use_gpu, use_mps, data_dir, labels_csv, weight_decay, max_patience, batch_size, label_smoothing):
    device = get_device(use_gpu, use_mps)
    print(f"Using device: {device}")
    train_loader, valid_loader = load_data3(device, data_dir, labels_csv, batch_size=batch_size, label_smoothing=label_smoothing)
    
    model = MRNet3()
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=max_patience, factor=.3, threshold=1e-4)

    best_val_auc = float('-inf')

    start_time = datetime.now()

    for epoch in range(epochs):
        change = datetime.now() - start_time
        print('starting epoch {}. time passed: {}'.format(epoch+1, str(change)))
        
        train_loss, train_auc, _, _ = run_model(model, train_loader, train=True, optimizer=optimizer)
        print(f'train loss: {train_loss:0.4f}')
        print(f'train AUC: {train_auc:0.4f}')

        val_loss, val_auc, _, _ = run_model(model, valid_loader, train=False)
        print(f'valid loss: {val_loss:0.4f}')
        print(f'valid AUC: {val_auc:0.4f}')

        scheduler.step(val_loss)

        if val_auc > best_val_auc:
            best_val_auc = val_auc

            file_name = f'val{val_auc:0.4f}_train{train_auc:0.4f}_epoch{epoch+1}'
            save_path = Path(rundir) / file_name
            
            print(f"Saving model to {save_path}")
            
            torch.save(model.state_dict(), save_path)



In [ ]:
import argparse
import matplotlib.pyplot as plt
import os
import numpy as np
import torch

from sklearn import metrics
from torch.autograd import Variable
from tqdm import tqdm
from torch.cuda.amp import autocast

def get_device(use_gpu, use_mps):
    if use_gpu and torch.cuda.is_available():
        return torch.device("cuda")
    elif use_mps and torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")

def run_model(model, loader, train=False, optimizer=None):
    preds = []
    labels = []

    if train:
        model.train()
    else:
        model.eval()

    total_loss = 0.
    num_batches = 0
    print(f"num_batches: {len(loader)}")
    for batch in tqdm(loader, desc="Processing batches", total=len(loader)):
        if train:
            optimizer.zero_grad()

        vol, label, original_slices = batch
        
        vol_device = vol  # List of [B, S_max, 3, 224, 224]
        label = label.to(loader.dataset.device)

        if str(loader.dataset.device).startswith('cuda'):
            with autocast(enabled=True):
                logit = model.forward(vol_device, original_slices)
                loss = loader.dataset.weighted_loss(logit, label, train)
        else:
            logit = model.forward(vol_device, original_slices)
            loss = loader.dataset.weighted_loss(logit, label, train)
        
        total_loss += loss.item()

        pred = torch.sigmoid(logit)
        pred_npy = pred.data.cpu().numpy().flatten()
        label_npy = label.data.cpu().numpy().flatten()

        preds.extend(pred_npy)
        labels.extend(label_npy)

        if train:
            loss.backward()
            optimizer.step()
        num_batches += 1

    avg_loss = total_loss / num_batches

    fpr, tpr, threshold = metrics.roc_curve(labels, preds)
    auc = metrics.auc(fpr, tpr)

    return avg_loss, auc, preds, labels

def evaluate(split, model_path, use_gpu, use_mps, data_dir, labels_csv, batch_size, label_smoothing):
    device = get_device(use_gpu, use_mps)
    print(f"Using device: {device}")
    
    if split == 'train' or split == 'valid':
        train_loader, valid_loader = load_data3(device, data_dir, labels_csv, batch_size=batch_size, label_smoothing=label_smoothing)

    elif split == 'test':
        test_loader = load_data_test(device, data_dir, labels_csv, batch_size=batch_size, label_smoothing=label_smoothing)

    else:
        raise ValueError("split must be 'train', 'valid', or 'test'")
    
    print("Loading model from path:", model_path)

    model = MRNet3()
    state_dict = torch.load(model_path, map_location=device)
    model.load_state_dict(state_dict)
    model = model.to(device)

    if split == 'train':
        loader = train_loader
    elif split == 'valid':
        loader = valid_loader
    elif split == 'test':
        loader = test_loader

    loss, auc, preds, labels = run_model(model, loader, train=False)

    print(f'{split} loss: {loss:0.4f}')
    print(f'{split} AUC: {auc:0.4f}')

    return preds, labels

<h3>Training</h3>


In [ ]:

rundir =  "/Users/matteobruno/Desktop/runs"  #"directory/to/store/runs"
data_dir = "/Users/matteobruno/Desktop/MRNet-v1.0/train" #"Directory/containing/.npy_files'"
labels_csv =  "/Users/matteobruno/Desktop/MRNet-v1.0/train/train-acl.csv" #"Path/to/labels/CSV/file"
seed = 42
gpu = False #If true runs on Nvidia GPU
mps = True #If true runs on Apple MPS
learning_rate = 1e-04
weight_decay = 5e-04
epochs = 50
max_patience = 5
batch_size = 4
label_smoothing = 0.1 #Label smoothing factor (0.0 = no smoothing)'


np.random.seed(seed)
torch.manual_seed(seed)
if gpu and torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
elif mps and torch.backends.mps.is_available():
    pass

os.makedirs(rundir, exist_ok=True)

# Save parameters to args.json
params = {
    "rundir": rundir,
    "data_dir": data_dir,
    "labels_csv": labels_csv,
    "seed": seed,
    "gpu": gpu,
    "mps": mps,
    "learning_rate": learning_rate,
    "weight_decay": weight_decay,
    "epochs": epochs,
    "max_patience": max_patience,
    "batch_size": batch_size,
    "label_smoothing": label_smoothing,
}
with open(Path(rundir) / 'args.json', 'w') as out:
    json.dump(params, out, indent=4)
    
    train3(rundir, epochs, learning_rate, gpu, mps, data_dir, labels_csv, weight_decay, 
           max_patience, batch_size, label_smoothing)

<h3>Testing</h3>


In [ ]:
model_path = "/Users/matteobruno/Desktop/Best_resnet/val0.9547_train0.9974_epoch26"  # Path to the saved model
split = "test"  # or "train", "valid"
data_dir = "/Users/matteobruno/Desktop/MRNet-v1.0/test" #"Directory/containing/.npy_files'"
labels_csv =  "/Users/matteobruno/Desktop/MRNet-v1.0/test/valid-acl.csv" #"Path/to/labels/CSV/file"
gpu = False #If true runs on Nvidia GPU
mps = True #If true runs on Apple MPS
batch_size = 1
label_smoothing = 0.0 #Label smoothing factor (0.0 = no smoothing)'

evaluate(split, model_path, gpu, mps, data_dir, labels_csv, batch_size, label_smoothing)

<h1>Efficientnet</h1>
Even after trying a lot of different hyperparamenters (trying different values for learnign rate, weight decay, batch size, dropout and trying to run the model with and without data augmentation) we didn't manage to get a good model.
We include it here for completeness 

<h3>Model class</h3>

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision.models import EfficientNet_B0_Weights

class MRNet3(nn.Module):
    def __init__(self):
        super().__init__()
            
        # Load pretrained EfficientNet-B0 from torchvision
        self.model1 = models.efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
        self.model2 = models.efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
        self.model3 = models.efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
        
        # Remove the classifier head to get feature extractor
        self.model1.classifier = nn.Identity()  # EfficientNet-B0 outputs 1280 features
        self.model2.classifier = nn.Identity()
        self.model3.classifier = nn.Identity()
        
        # Enhanced fully connected classifier
        self.classifier = nn.Sequential(
            nn.Linear(1280 * 3, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 1)
            )

    def forward(self, x, original_slices):
        view_features = []
        
        for view in range(3):
            x_view = x[view]  # [B, S_max, 3, 224, 224]
            B, S_max, _, H, W = x_view.shape
            x_view = x_view.view(B * S_max, 3, H, W)
            
            if view == 0:
                features = self.model1(x_view)  # [B * S_max, 1280]
            elif view == 1:
                features = self.model2(x_view)
            else:
                features = self.model3(x_view)
            
            features = features.view(B, S_max, 1280)  # [B, S_max, 1280]
            s_indices = torch.arange(S_max, device=features.device).unsqueeze(0).expand(B, S_max)
            mask = s_indices < original_slices[view].unsqueeze(1)
            features = features.masked_fill(~mask.unsqueeze(2), -float('inf'))
            max_features = torch.max(features, dim=1)[0]  # [B, 1280]
            
            view_features.append(max_features)
        
        # Concatenate features from all views
        x_stacked = torch.cat(view_features, dim=1)  # [B, 1280 * 3 = 3840]
        
        # Pass through the enhanced classifier
        output = self.classifier(x_stacked)  # [B, 1]
        
        return output

<h3>Loader</h3>

In [ ]:
import numpy as np
import os
import torch
import torch.nn.functional as F
import torch.utils.data as data
import pandas as pd
from sklearn.model_selection import train_test_split

INPUT_DIM = 224
MAX_PIXEL_VAL = 1.0  
MEAN = [0.485, 0.456, 0.406] 
STDDEV = [0.229, 0.224, 0.225]  

class MRDataset(data.Dataset):
    def __init__(self, data_dir, file_list, labels_dict, device, label_smoothing=0.1):
        super().__init__()
        self.device = device
        self.data_dir_axial = f"{data_dir}/axial"
        self.data_dir_coronal = f"{data_dir}/coronal"
        self.data_dir_sagittal = f"{data_dir}/sagittal"

        self.paths_axial = [os.path.join(self.data_dir_axial, file) for file in file_list]
        self.paths_coronal = [os.path.join(self.data_dir_coronal, file) for file in file_list]
        self.paths_sagittal = [os.path.join(self.data_dir_sagittal, file) for file in file_list]
        
        self.paths = [self.paths_axial, self.paths_coronal, self.paths_sagittal]
        
        self.labels = [labels_dict[file] for file in file_list]
        self.label_smoothing = label_smoothing  # New parameter for label smoothing

        neg_weight = np.mean(self.labels)
        dtype = np.float32
        self.weights = [dtype(neg_weight), dtype(1 - neg_weight)]

    def weighted_loss(self, prediction, target, train):
        dtype = torch.float32
        indices = target.squeeze(1).long()  # Shape: [B]
        weights_tensor = torch.tensor(self.weights, device=self.device, dtype=dtype)[indices]  # Shape: [B]
        weights_tensor = weights_tensor.unsqueeze(1)  # Shape: [B, 1]

        # Apply label smoothing only during training if label_smoothing > 0
        if train and self.label_smoothing > 0:
            smoothed_target = target * (1 - self.label_smoothing) + (1 - target) * self.label_smoothing
        else:
            smoothed_target = target

        loss = F.binary_cross_entropy_with_logits(prediction, smoothed_target, weight=weights_tensor)
        return loss

    def __getitem__(self, index):
        vol_list = []

        for i in range(3):           
            path = self.paths[i][index]
            vol = np.load(path).astype(np.float32) 

            # Crop to INPUT_DIM x INPUT_DIM (224x224)
            pad = int((vol.shape[2] - INPUT_DIM) / 2)
            vol = vol[:, pad:-pad, pad:-pad]

            # Normalize to [0, 1]
            vol = (vol - np.min(vol)) / (np.max(vol) - np.min(vol) + 1e-6)  # [0, 1]

            # Stack to 3 channels
            vol = np.stack((vol,) * 3, axis=1)  # Shape: (slices, 3, 224, 224)

            # Apply ImageNet normalization per channel
            vol_tensor = torch.FloatTensor(vol).to(self.device)  # Shape: (slices, 3, 224, 224)
            for c in range(3):
                vol_tensor[:, c, :, :] = (vol_tensor[:, c, :, :] - MEAN[c]) / STDDEV[c]

            vol_list.append(vol_tensor)

        label_tensor = torch.FloatTensor([self.labels[index]]).to(self.device)

        return vol_list, label_tensor

    def __len__(self):
        return len(self.labels)

def collate_fn(batch):
    device = batch[0][0][0].device
    view0_list = [sample[0][0] for sample in batch]  # Axial
    view1_list = [sample[0][1] for sample in batch]  # Coronal
    view2_list = [sample[0][2] for sample in batch]  # Sagittal
    
    # Pad slices to the maximum in the batch for each view
    padded_view0 = torch.nn.utils.rnn.pad_sequence(view0_list, batch_first=True)
    padded_view1 = torch.nn.utils.rnn.pad_sequence(view1_list, batch_first=True)
    padded_view2 = torch.nn.utils.rnn.pad_sequence(view2_list, batch_first=True)
    
    # Store original slice counts for masking in the model
    original_slices0 = torch.tensor([v.shape[0] for v in view0_list], device=device)
    original_slices1 = torch.tensor([v.shape[0] for v in view1_list], device=device)
    original_slices2 = torch.tensor([v.shape[0] for v in view2_list], device=device)
    
    # Stack labels
    labels = torch.stack([sample[1] for sample in batch])
    
    return [padded_view0, padded_view1, padded_view2], labels, [original_slices0, original_slices1, original_slices2]

def load_data3(device, data_dir, labels_csv, batch_size=1, label_smoothing=0.1):
    labels_df = pd.read_csv(labels_csv, header=None, names=['filename', 'label'])
    labels_df['filename'] = labels_df['filename'].apply(lambda x: f"{int(x):04d}.npy")
    labels_dict = dict(zip(labels_df['filename'], labels_df['label']))

    all_files = [f for f in os.listdir(f"{data_dir}/axial") if f.endswith(".npy")]
    all_files = [f for f in all_files if f in labels_dict]
    all_files.sort()

    labels = [labels_dict[file] for file in all_files]

    train_files, valid_files = train_test_split(
        all_files, 
        test_size=0.2, 
        random_state=42, 
        stratify=labels
    )

    train_dataset = MRDataset(data_dir, train_files, labels_dict, device, label_smoothing=label_smoothing)
    valid_dataset = MRDataset(data_dir, valid_files, labels_dict, device, label_smoothing=label_smoothing)

    train_loader = data.DataLoader(train_dataset, batch_size=batch_size, num_workers=0, shuffle=True, collate_fn=collate_fn)
    valid_loader = data.DataLoader(valid_dataset, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=collate_fn)

    return train_loader, valid_loader

def load_data_test(device, data_dir, labels_csv, batch_size=1, label_smoothing=0):
    
    labels_df = pd.read_csv(labels_csv, header=None, names=['filename', 'label'])
    labels_df['filename'] = labels_df['filename'].apply(lambda x: f"{int(x):04d}.npy")
    labels_dict = dict(zip(labels_df['filename'], labels_df['label']))

    test_files = [f for f in os.listdir(f"{data_dir}/axial") if f.endswith(".npy")]
    test_files = [f for f in test_files if f in labels_dict]
    test_files.sort()

    test_dataset = MRDataset(data_dir, test_files, labels_dict, device, label_smoothing=label_smoothing)

    test_loader = data.DataLoader(test_dataset, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=collate_fn)

    return test_loader

<h3>Training and evaluation functions</h3>

In [ ]:
import argparse
import matplotlib.pyplot as plt
import os
import numpy as np
import torch

from sklearn import metrics
from torch.autograd import Variable
from tqdm import tqdm
from torch.cuda.amp import autocast

def get_device(use_gpu, use_mps):
    if use_gpu and torch.cuda.is_available():
        return torch.device("cuda")
    elif use_mps and torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")

def run_model(model, loader, train=False, optimizer=None, accumulation_steps=4):
    """
    Run the model on the given data loader with gradient accumulation for training.
    
    Args:
        model: The neural network model.
        loader: DataLoader providing the batches.
        train: Boolean indicating training or evaluation mode.
        optimizer: Optimizer used for weight updates (required if train=True).
        accumulation_steps: Number of batches to accumulate gradients over (default: 4).
    
    Returns:
        avg_loss: Average loss over the dataset.
        auc: Area under the ROC curve.
        preds: List of predictions.
        labels: List of true labels.
    """
    preds = []
    labels = []
    total_loss = 0.
    num_batches = 0

    # Set model mode
    if train:
        model.train()
    else:
        model.eval()

    # Process batches
    for i, batch in enumerate(tqdm(loader, desc="Processing batches", total=len(loader))):
        vol, label, original_slices = batch
        vol_device = vol  # Assuming vol is already on the correct device
        label = label.to(loader.dataset.device)

        # Zero gradients at the start of an accumulation cycle
        if train and i % accumulation_steps == 0:
            optimizer.zero_grad()

        # Forward pass
        if str(loader.dataset.device).startswith('cuda'):
            with autocast(enabled=True):  # Mixed precision for CUDA
                logit = model.forward(vol_device, original_slices)
                loss = loader.dataset.weighted_loss(logit, label, train) / accumulation_steps
        else:
            logit = model.forward(vol_device, original_slices)
            loss = loader.dataset.weighted_loss(logit, label, train) / accumulation_steps

        # Backward pass for training
        if train:
            loss.backward()  # Accumulate gradients

            # Update weights after accumulation_steps batches
            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

        # Track loss (scale back for logging)
        total_loss += loss.item() * accumulation_steps
        num_batches += 1

        # Collect predictions and labels
        pred = torch.sigmoid(logit)
        pred_npy = pred.data.cpu().numpy().flatten()
        label_npy = label.data.cpu().numpy().flatten()
        preds.extend(pred_npy)
        labels.extend(label_npy)

    # Handle remaining gradients at the end of the epoch
    if train and num_batches % accumulation_steps != 0:
        optimizer.step()
        optimizer.zero_grad()

    # Compute average loss and AUC
    avg_loss = total_loss / num_batches
    fpr, tpr, _ = metrics.roc_curve(labels, preds)
    auc = metrics.auc(fpr, tpr)

    return avg_loss, auc, preds, labels

def evaluate(split, model_path, use_gpu, use_mps, data_dir, labels_csv, batch_size, label_smoothing):
    device = get_device(use_gpu, use_mps)
    print(f"Using device: {device}")
    
    if split == 'train' or split == 'valid':
        train_loader, valid_loader = load_data3(device, data_dir, labels_csv, batch_size=batch_size, label_smoothing=label_smoothing)

    elif split == 'test':
        test_loader = load_data_test(device, data_dir, labels_csv, batch_size=batch_size, label_smoothing=label_smoothing)

    else:
        raise ValueError("split must be 'train', 'valid', or 'test'")
    
    print("Loading model from path:", model_path)

    model = MRNet3()
    state_dict = torch.load(model_path, map_location=device)
    model.load_state_dict(state_dict)
    model = model.to(device)

    if split == 'train':
        loader = train_loader
    elif split == 'valid':
        loader = valid_loader
    elif split == 'test':
        loader = test_loader

    loss, auc, preds, labels = run_model(model, loader, train=False)

    print(f'{split} loss: {loss:0.4f}')
    print(f'{split} AUC: {auc:0.4f}')

    return preds, labels

In [ ]:
import argparse
import json
import numpy as np
import os
import torch

from datetime import datetime
from pathlib import Path
from sklearn import metrics

def get_device(use_gpu, use_mps):
    if use_gpu and torch.cuda.is_available():
        return torch.device("cuda")
    elif use_mps and torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")

def train3(rundir, epochs, learning_rate, use_gpu, use_mps, data_dir, labels_csv, weight_decay, max_patience, batch_size, label_smoothing):
    device = get_device(use_gpu, use_mps)
    print(f"Using device: {device}")
    train_loader, _ = load_data3(device, data_dir, labels_csv, batch_size=batch_size, label_smoothing=label_smoothing)
    _, valid_loader = load_data3(device, data_dir, labels_csv, batch_size=1, label_smoothing=label_smoothing)

    model = MRNet3()
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=max_patience, factor=.3, threshold=1e-4)

    best_val_auc = float('-inf')

    start_time = datetime.now()

    for epoch in range(epochs):
        change = datetime.now() - start_time
        print('starting epoch {}. time passed: {}'.format(epoch+1, str(change)))
        
        train_loss, train_auc, _, _ = run_model(model, train_loader, train=True, optimizer=optimizer)
        print(f'train loss: {train_loss:0.4f}')
        print(f'train AUC: {train_auc:0.4f}')

        val_loss, val_auc, _, _ = run_model(model, valid_loader, train=False)
        print(f'valid loss: {val_loss:0.4f}')
        print(f'valid AUC: {val_auc:0.4f}')

        scheduler.step(val_loss)

        if val_auc > best_val_auc:
            best_val_auc = val_auc

            file_name = f'val{val_auc:0.4f}_train{train_auc:0.4f}_epoch{epoch+1}'
            save_path = Path(rundir) / file_name
            
            print(f"Saving model to {save_path}")
            
            torch.save(model.state_dict(), save_path)



<h3>Training</h3>

In [ ]:

rundir =  "/Users/matteobruno/Desktop/runs"  #"directory/to/store/runs"
data_dir = "/Users/matteobruno/Desktop/MRNet-v1.0/train" #"Directory/containing/.npy_files'"
labels_csv =  "/Users/matteobruno/Desktop/MRNet-v1.0/train/train-acl.csv" #"Path/to/labels/CSV/file"
seed = 42
gpu = False #If true runs on Nvidia GPU
mps = True #If true runs on Apple MPS
learning_rate = 1e-04
weight_decay = 1e-05
epochs = 50
max_patience = 5
batch_size = 4
label_smoothing = 0.0 #Label smoothing factor (0.0 = no smoothing)'


np.random.seed(seed)
torch.manual_seed(seed)
if gpu and torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
elif mps and torch.backends.mps.is_available():
    pass

os.makedirs(rundir, exist_ok=True)


# Save parameters to args.json
params = {
    "rundir": rundir,
    "data_dir": data_dir,
    "labels_csv": labels_csv,
    "seed": seed,
    "gpu": gpu,
    "mps": mps,
    "learning_rate": learning_rate,
    "weight_decay": weight_decay,
    "epochs": epochs,
    "max_patience": max_patience,
    "batch_size": batch_size,
    "label_smoothing": label_smoothing,
}
with open(Path(rundir) / 'args.json', 'w') as out:
    json.dump(params, out, indent=4)
    
    train3(rundir, epochs, learning_rate, gpu, mps, data_dir, labels_csv, weight_decay, 
           max_patience, batch_size, label_smoothing)

<h3>Testing</h3>


In [ ]:
model_path = "/Users/matteobruno/Desktop/"  # Path to the saved model
split = "test"  # or "train", "valid"
data_dir = "/Users/matteobruno/Desktop/MRNet-v1.0/test" #"Directory/containing/.npy_files'"
labels_csv =  "/Users/matteobruno/Desktop/MRNet-v1.0/test/valid-acl.csv" #"Path/to/labels/CSV/file"
gpu = False #If true runs on Nvidia GPU
mps = True #If true runs on Apple MPS
batch_size = 1
label_smoothing = 0.0 #Label smoothing factor (0.0 = no smoothing)'

evaluate(split, model_path, gpu, mps, data_dir, labels_csv, batch_size, label_smoothing)

<h1> Ensemble model</h1>
Finally we decide to build an ensemble model with the two best model we got, that is Alexnet Majority Vote and Resnet. 
In this model each sample goes trough the Resent-based model convolutoinal layers and trough the ALexnet-based model convolutional layers. The two resulting feature vector are then concatenated and passed trough a three layers beep MLP to obtain the final probabilities.
During training the weigths from the best Alexnet-based and Resnet-based models are loaded for the convolutional layers, which are then frozen. This allows us to only train the MLP layers which makes the task feasible on teh available hardware.
A small performance from the Resnet model increase was obtained, with a validation AUC of 96.21% (train AUC of 98.25) and a testing AUC of 96.38%

<h3>Model class</h3>

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision.models import AlexNet_Weights, ResNet18_Weights

class MRNetAlex(nn.Module):
    """Model 1: AlexNet-based model with separate classifiers per view."""
    def __init__(self, use_batchnorm=False):
        super().__init__()
        self.model1 = models.alexnet(weights=AlexNet_Weights.DEFAULT)  # Axial
        self.model2 = models.alexnet(weights=AlexNet_Weights.DEFAULT)  # Coronal
        self.model3 = models.alexnet(weights=AlexNet_Weights.DEFAULT)  # Sagittal
        self.gap = nn.AdaptiveMaxPool2d(1)
        self.use_batchnorm = use_batchnorm
        n = 0.15
        self.dropout_view1 = nn.Dropout(p=n)
        self.dropout_view2 = nn.Dropout(p=n)
        self.dropout_view3 = nn.Dropout(p=n)

        # Classifiers for each view
        classifier_layers_axial = [nn.Linear(256, 256)]
        if self.use_batchnorm:
            classifier_layers_axial.append(nn.BatchNorm1d(256))
        self.classifier1_axial = nn.Sequential(*classifier_layers_axial)
        self.classifier1_coronal = nn.Sequential(*[nn.Linear(256, 256)] + ([nn.BatchNorm1d(256)] if self.use_batchnorm else []))
        self.classifier1_sagittal = nn.Sequential(*[nn.Linear(256, 256)] + ([nn.BatchNorm1d(256)] if self.use_batchnorm else []))
        self.classifier2_axial = nn.Linear(256, 1)
        self.classifier2_coronal = nn.Linear(256, 1)
        self.classifier2_sagittal = nn.Linear(256, 1)

    def forward(self, x):
        # Not implemented as it's not needed for the ensemble
        pass

class MRNetResNet(nn.Module):
    """Model 2: ResNet18-based model with feature concatenation."""
    def __init__(self):
        super().__init__()
        self.model1 = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        self.model1 = nn.Sequential(*list(self.model1.children())[:-1])  # Axial
        self.model2 = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        self.model2 = nn.Sequential(*list(self.model2.children())[:-1])  # Coronal
        self.model3 = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        self.model3 = nn.Sequential(*list(self.model3.children())[:-1])  # Sagittal
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.dropout_view1 = nn.Dropout(p=0.7)
        self.dropout_view2 = nn.Dropout(p=0.7)
        self.dropout_view3 = nn.Dropout(p=0.7)
        self.classifier1 = nn.Linear(512 * 3, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.4)
        self.activation = nn.ReLU()
        self.classifier2 = nn.Linear(256, 1)

    def forward(self, x, original_slices):
        # Not implemented as it's not needed for the ensemble
        pass

class EnsembleMRNet(nn.Module):
    """Ensemble model combining CNNs from MRNetAlex and MRNetResNet with a new dense classifier."""
    def __init__(self, model1_path=None, model2_path=None, device="cpu"):
        super().__init__()
        # Initialize base models
        self.model_alex = MRNetAlex()
        self.model_resnet = MRNetResNet()
        
        # Load pre-trained weights
        if model1_path is not None:
            self.model_alex.load_state_dict(torch.load(model1_path, map_location=device, weights_only=True))
        if model2_path is not None:
            self.model_resnet.load_state_dict(torch.load(model2_path, map_location=device, weights_only=True))
        
        # Freeze CNN parts
        for model in [self.model_alex.model1, self.model_alex.model2, self.model_alex.model3]:
            for param in model.features.parameters():
                param.requires_grad = False
        for model in [self.model_resnet.model1, self.model_resnet.model2, self.model_resnet.model3]:
            for param in model.parameters():
                param.requires_grad = False
        
        # Define pooling layers consistent with original models
        self.gap_max = nn.AdaptiveMaxPool2d(1)  # For AlexNet
        self.gap_avg = nn.AdaptiveAvgPool2d(1)  # For ResNet18
        
        # Define dropout layers for each backbone per view
        self.dropout_alex_view1 = nn.Dropout(p=0.15)
        self.dropout_alex_view2 = nn.Dropout(p=0.15)
        self.dropout_alex_view3 = nn.Dropout(p=0.15)
        self.dropout_resnet_view1 = nn.Dropout(p=0.7)
        self.dropout_resnet_view2 = nn.Dropout(p=0.7)
        self.dropout_resnet_view3 = nn.Dropout(p=0.7)
        
        # New dense classifier
        self.dense = nn.Sequential(
            nn.Linear(2304, 1024),  # Input: 2304, Output: 512           
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            ###wasn't there in the best model 
            nn.Linear(1024, 512),   # Input: 1024, Output: 512
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            ###wasn't there in the best model
            nn.Linear(512, 1)      # Output: 1 for binary classification
        )

    def forward(self, padded_views, original_slices):
        """Forward pass: Extract features from both CNNs, concatenate, and classify."""
        B = padded_views[0].shape[0]
        view_features = []
        
        for view in range(3):
            x_view = padded_views[view]  # [B, S_max, 3, 224, 224]
            S_max = x_view.shape[1]
            x_view_flat = x_view.view(B * S_max, 3, 224, 224)
            
            # AlexNet features
            if view == 0:
                feat_alex = self.model_alex.model1.features(x_view_flat)  # [B*S_max, 256, 6, 6]
            elif view == 1:
                feat_alex = self.model_alex.model2.features(x_view_flat)
            else:
                feat_alex = self.model_alex.model3.features(x_view_flat)
            feat_alex = self.gap_max(feat_alex).view(B, S_max, 256)
            mask = torch.arange(S_max, device=feat_alex.device).expand(B, S_max) < original_slices[view].unsqueeze(1)
            feat_alex = feat_alex.masked_fill(~mask.unsqueeze(2), -float('inf'))
            max_feat_alex = torch.max(feat_alex, dim=1)[0]  # [B, 256]
            
            # ResNet18 features
            if view == 0:
                feat_resnet = self.model_resnet.model1(x_view_flat)  # [B*S_max, 512, 7, 7]
            elif view == 1:
                feat_resnet = self.model_resnet.model2(x_view_flat)
            else:
                feat_resnet = self.model_resnet.model3(x_view_flat)
            feat_resnet = self.gap_avg(feat_resnet).view(B, S_max, 512)
            feat_resnet = feat_resnet.masked_fill(~mask.unsqueeze(2), -float('inf'))
            max_feat_resnet = torch.max(feat_resnet, dim=1)[0]  # [B, 512]
            
            # Apply dropout
            if view == 0:
                max_feat_alex = self.dropout_alex_view1(max_feat_alex)
                max_feat_resnet = self.dropout_resnet_view1(max_feat_resnet)
            elif view == 1:
                max_feat_alex = self.dropout_alex_view2(max_feat_alex)
                max_feat_resnet = self.dropout_resnet_view2(max_feat_resnet)
            else:
                max_feat_alex = self.dropout_alex_view3(max_feat_alex)
                max_feat_resnet = self.dropout_resnet_view3(max_feat_resnet)
            
            # Concatenate features for this view
            combined_feat = torch.cat([max_feat_alex, max_feat_resnet], dim=1)  # [B, 768]
            view_features.append(combined_feat)
        
        # Concatenate all views
        all_features = torch.cat(view_features, dim=1)  # [B, 2304]
        
        # Dense classifier
        logits = self.dense(all_features)  # [B, 1]
        return logits

<h3>Loader</h3>

In [ ]:
import numpy as np
import os
import torch
import torch.nn.functional as F
import torch.utils.data as data
import pandas as pd
from sklearn.model_selection import train_test_split
import kornia.augmentation as K
import random

INPUT_DIM = 224
MAX_PIXEL_VAL = 1.0
MEAN = [0.485, 0.456, 0.406]
STDDEV = [0.229, 0.224, 0.225]

class MRDataset(data.Dataset):
    def __init__(self, data_dir, file_list, labels_dict, device, label_smoothing=0.1, is_training=False):
        super().__init__()
        self.device = device
        self.is_training = is_training
        self.data_dir_axial = f"{data_dir}/axial"
        self.data_dir_coronal = f"{data_dir}/coronal"
        self.data_dir_sagittal = f"{data_dir}/sagittal"
        self.paths_axial = [os.path.join(self.data_dir_axial, file) for file in file_list]
        self.paths_coronal = [os.path.join(self.data_dir_coronal, file) for file in file_list]
        self.paths_sagittal = [os.path.join(self.data_dir_sagittal, file) for file in file_list]
        self.paths = [self.paths_axial, self.paths_coronal, self.paths_sagittal]
        self.labels = [labels_dict[file] for file in file_list]
        self.label_smoothing = label_smoothing
        neg_weight = np.mean(self.labels)
        self.weights = [float(neg_weight), float(1 - neg_weight)]

    def weighted_loss(self, prediction, target, train):
        indices = target.squeeze(1).long()
        weights_tensor = torch.tensor(self.weights, device=self.device, dtype=torch.float32)[indices].unsqueeze(1)
        smoothed_target = target * (1 - self.label_smoothing) + (1 - target) * self.label_smoothing if train and self.label_smoothing > 0 else target
        loss = F.binary_cross_entropy_with_logits(prediction, smoothed_target, weight=weights_tensor)
        return loss

    def __getitem__(self, index):
        vol_list = []
        for i in range(3):
            path = self.paths[i][index]
            vol = np.load(path).astype(np.float32)
            pad = int((vol.shape[2] - INPUT_DIM) / 2)
            vol = vol[:, pad:-pad, pad:-pad]
            vol = (vol - np.min(vol)) / (np.max(vol) - np.min(vol) + 1e-6)
            vol = np.stack((vol,) * 3, axis=1)
            vol_tensor = torch.FloatTensor(vol).to(self.device)
            
            # Apply augmentations only during training
            if self.is_training and random.random() < 0.3:
                vol_tensor = self.apply_augmentations(vol_tensor)

            for c in range(3):
                vol_tensor[:, c, :, :] = (vol_tensor[:, c, :, :] - MEAN[c]) / STDDEV[c]
            vol_list.append(vol_tensor)
        label_tensor = torch.FloatTensor([self.labels[index]]).to(self.device)
        return vol_list, label_tensor

    def apply_augmentations(self, vol_tensor):
        # vol_tensor shape: [slices, channels, height, width]
        # Reshape to treat slices as batch dimension
        vol_tensor = vol_tensor.permute(1, 0, 2, 3)  # [channels, slices, height, width]
        
        # Apply augmentations
        vol_tensor = K.RandomRotation(degrees=25, keepdim=True)(vol_tensor)
        vol_tensor = K.RandomAffine(degrees=0, translate=(25/224, 25/224), keepdim=True)(vol_tensor)
        if random.random() > 0.5:
            vol_tensor = K.RandomHorizontalFlip(p=1.0, keepdim=True)(vol_tensor)
        
        # Reshape back to [slices, channels, height, width]
        vol_tensor = vol_tensor.permute(1, 0, 2, 3)
        return vol_tensor

    def __len__(self):
        return len(self.labels)

def collate_fn(batch):
    device = batch[0][0][0].device
    view0_list = [sample[0][0] for sample in batch]
    view1_list = [sample[0][1] for sample in batch]
    view2_list = [sample[0][2] for sample in batch]
    padded_view0 = torch.nn.utils.rnn.pad_sequence(view0_list, batch_first=True)
    padded_view1 = torch.nn.utils.rnn.pad_sequence(view1_list, batch_first=True)
    padded_view2 = torch.nn.utils.rnn.pad_sequence(view2_list, batch_first=True)
    original_slices0 = torch.tensor([v.shape[0] for v in view0_list], device=device)
    original_slices1 = torch.tensor([v.shape[0] for v in view1_list], device=device)
    original_slices2 = torch.tensor([v.shape[0] for v in view2_list], device=device)
    labels = torch.stack([sample[1] for sample in batch])
    return [padded_view0, padded_view1, padded_view2], labels, [original_slices0, original_slices1, original_slices2]

def load_data3(device, data_dir, labels_csv, batch_size=1, label_smoothing=0.1):
    labels_df = pd.read_csv(labels_csv, header=None, names=['filename', 'label'])
    labels_df['filename'] = labels_df['filename'].apply(lambda x: f"{int(x):04d}.npy")
    labels_dict = dict(zip(labels_df['filename'], labels_df['label']))
    all_files = [f for f in os.listdir(f"{data_dir}/axial") if f.endswith(".npy") and f in labels_dict]
    all_files.sort()
    labels = [labels_dict[file] for file in all_files]
    train_files, valid_files = train_test_split(
        all_files, test_size=0.2, random_state=42, stratify=labels
    )
    train_dataset = MRDataset(data_dir, train_files, labels_dict, device, label_smoothing, is_training=True)
    valid_dataset = MRDataset(data_dir, valid_files, labels_dict, device, label_smoothing, is_training=False)
    train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=collate_fn)
    valid_loader = data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=collate_fn)
    return train_loader, valid_loader

def load_data_test(device, data_dir, labels_csv, batch_size=1, label_smoothing=0.1):
    labels_df = pd.read_csv(labels_csv, header=None, names=['filename', 'label'])
    labels_df['filename'] = labels_df['filename'].apply(lambda x: f"{int(x):04d}.npy")
    labels_dict = dict(zip(labels_df['filename'], labels_df['label']))
    test_files = [f for f in os.listdir(f"{data_dir}/axial") if f.endswith(".npy") and f in labels_dict]
    test_files.sort()
    test_dataset = MRDataset(data_dir, test_files, labels_dict, device, label_smoothing, is_training=False)
    test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=collate_fn)
    return test_loader

<h3>Training and evaluation functions</h3>

In [ ]:
import argparse
import matplotlib.pyplot as plt
import os
import numpy as np
import torch
from sklearn import metrics
from torch.autograd import Variable
from tqdm import tqdm
from torch.amp import autocast

def get_device(use_gpu, use_mps):
    if use_gpu and torch.cuda.is_available():
        return torch.device("cuda")
    elif use_mps and torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")

def run_model(model, loader, train=False, optimizer=None):
    preds = []
    labels = []
    model.train() if train else model.eval()
    total_loss = 0.0
    num_batches = 0
    for batch in tqdm(loader, desc="Processing batches", total=len(loader)):
        if train:
            optimizer.zero_grad()
        padded_views, label, original_slices = batch
        label = label.to(loader.dataset.device)
        if str(loader.dataset.device).startswith('cuda'):
            with autocast(device_type='cuda', enabled=True):
                logit = model(padded_views, original_slices)
                loss = loader.dataset.weighted_loss(logit, label, train)
        else:
            logit = model(padded_views, original_slices)
            loss = loader.dataset.weighted_loss(logit, label, train)
        total_loss += loss.item()
        pred = torch.sigmoid(logit)
        pred_npy = pred.data.cpu().numpy().flatten()
        label_npy = label.data.cpu().numpy().flatten()
        preds.extend(pred_npy)
        labels.extend(label_npy)
        if train:
            loss.backward()
            optimizer.step()
        num_batches += 1
    avg_loss = total_loss / num_batches
    fpr, tpr, _ = metrics.roc_curve(labels, preds)
    auc = metrics.auc(fpr, tpr)
    return avg_loss, auc, preds, labels

def evaluate(split, model_path, use_gpu, use_mps, data_dir, labels_csv, batch_size, label_smoothing):
    device = get_device(use_gpu, use_mps)
    print(f"Using device: {device}")
    if split in ['train', 'valid']:
        train_loader, valid_loader = load_data3(device, data_dir, labels_csv, batch_size=batch_size, label_smoothing=label_smoothing)
        loader = train_loader if split == 'train' else valid_loader
    elif split == 'test':
        loader = load_data_test(device, data_dir, labels_csv, batch_size=batch_size, label_smoothing=label_smoothing)
    else:
        raise ValueError("split must be 'train', 'valid', or 'test'")
    model = EnsembleMRNet(device=device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    loss, auc, preds, labels = run_model(model, loader, train=False)
    print(f'{split} loss: {loss:.4f}')
    print(f'{split} AUC: {auc:.4f}')
    return preds, labels

In [ ]:
import argparse
import json
import numpy as np
import os
import torch
from datetime import datetime
from pathlib import Path
from sklearn import metrics

def get_device(use_gpu, use_mps):
    if use_gpu and torch.cuda.is_available():
        return torch.device("cuda")
    elif use_mps and torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")

def train(rundir, model1_path, model2_path, epochs, learning_rate, use_gpu, use_mps, data_dir, labels_csv, weight_decay, max_patience, batch_size, label_smoothing):
    device = get_device(use_gpu, use_mps)
    print(f"Using device: {device}")
    train_loader, valid_loader = load_data3(device, data_dir, labels_csv, batch_size=batch_size, label_smoothing=label_smoothing)
    model = EnsembleMRNet(model1_path, model2_path, device).to(device)
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=max_patience, factor=0.3, threshold=1e-4)
    best_val_auc = float('-inf')
    start_time = datetime.now()
    
    for epoch in range(epochs):
        change = datetime.now() - start_time
        print(f'Starting epoch {epoch+1}. Time passed: {change}')
        train_loss, train_auc, _, _ = run_model(model, train_loader, train=True, optimizer=optimizer)
        print(f'Train loss: {train_loss:.4f}, Train AUC: {train_auc:.4f}')
        val_loss, val_auc, _, _ = run_model(model, valid_loader, train=False)
        print(f'Valid loss: {val_loss:.4f}, Valid AUC: {val_auc:.4f}')
        scheduler.step(val_loss)
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            file_name = f'val{val_auc:.4f}_train{train_auc:.4f}_epoch{epoch+1}'
            save_path = Path(rundir) / file_name
            torch.save(model.state_dict(), save_path)
            print(f"Model saved to {save_path}")

<h3>Training</h3>

In [ ]:

rundir =  "/Users/matteobruno/Desktop/runs"  #"directory/to/store/runs"
model1_path = "/Users/matteobruno/Desktop/Best_alexnet_majority/best_model.pth" #path to pre-trained AlexNet model
model2_path = "/Users/matteobruno/Desktop/Best_resnet/val0.9547_train0.9974_epoch26" #path to pre-trained ResNet model
data_dir = "/Users/matteobruno/Desktop/MRNet-v1.0/train" #"Directory/containing/.npy_files'"
labels_csv =  "/Users/matteobruno/Desktop/MRNet-v1.0/train/train-acl.csv" #"Path/to/labels/CSV/file"
seed = 42
gpu = False #If true runs on Nvidia GPU
mps = True #If true runs on Apple MPS
learning_rate = 1e-04
weight_decay = 1e-05
epochs = 50
max_patience = 5
batch_size = 4
label_smoothing = 0.0 #Label smoothing factor (0.0 = no smoothing)'

np.random.seed(seed)
torch.manual_seed(seed)
if gpu and torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
elif mps and torch.backends.mps.is_available():
    pass

os.makedirs(rundir, exist_ok=True)


# Save parameters to args.json
params = {
    "rundir": rundir,
    "model_path": model1_path,
    "model2_path": model2_path,
    "data_dir": data_dir,
    "labels_csv": labels_csv,
    "seed": seed,
    "gpu": gpu,
    "mps": mps,
    "learning_rate": learning_rate,
    "weight_decay": weight_decay,
    "epochs": epochs,
    "max_patience": max_patience,
    "batch_size": batch_size,
    "label_smoothing": label_smoothing,
}
with open(Path(rundir) / 'args.json', 'w') as out:
    json.dump(params, out, indent=4)
    
    train(rundir, model1_path, model2_path, epochs, learning_rate, gpu, mps, data_dir, labels_csv, weight_decay, 
           max_patience, batch_size, label_smoothing)

<h3>Testing</h3>

In [ ]:
model_path = "/Users/matteobruno/Desktop/Ensemble_best/val0.9621_train0.9895_epoch20"  # Path to the saved model
split = "test"  # or "train", "valid"
data_dir = "/Users/matteobruno/Desktop/MRNet-v1.0/test" #"Directory/containing/.npy_files'"
labels_csv =  "/Users/matteobruno/Desktop/MRNet-v1.0/test/valid-acl.csv" #"Path/to/labels/CSV/file"
gpu = False #If true runs on Nvidia GPU
mps = True #If true runs on Apple MPS
batch_size = 1
label_smoothing = 0.0 #Label smoothing factor (0.0 = no smoothing)'

evaluate(split, model_path, gpu, mps, data_dir, labels_csv, batch_size, label_smoothing)